0. Pré-requisitos

In [ ]:
from google.colab import drive
from datetime import datetime
import os

drive.mount('/content/drive', force_remount=True)
root_path = "/content/drive/My Drive/Colab Notebooks/data/"
temp_path = root_path + str(datetime.now())[:10].replace("-", ".")
if not os.path.exists(root_path):
  os.mkdir(root_path)
if not os.path.exists(temp_path):
  os.mkdir(temp_path)

Mounted at /content/drive


1. Recorra à API “Text Search” ou à API “Image Search” do Arquivo.pt para reunir um conjunto elevado de textos ou imagens que deverá guardar no seu computador.

2. Guarde os textos ou imagens na pasta /data/aaaa.mm.dd, onde aaaa.mm.dd é um valor que deverá ser dinamicamente obtido a partir da data de execução do código (e.g., 2020.05.26, no caso de o código ser executado no dia 26/05/2020). Proceda também à gravação das informações correspondentes (dos textos ou das imagens) num ficheiro JSON dentro da mesma pasta.

3. Carregue o ficheiro JSON em memória e percorra os conteúdos de 5 dos registos.

In [ ]:
import requests
import json
import concurrent.futures

max_workers=4

def download_image(image):
  try:
    filename = image["imgDigest"] + "_" + image["imgLinkToArchive"].split("/")[-1] # Unique name to the image
    if "?" in filename:
      filename = filename.split("?")[0]
    if filename in os.listdir(temp_path):
      return False
    img_response = requests.get(url=image["imgLinkToArchive"])
    if img_response.status_code < 200 or img_response.status_code > 299:
      return False
    with open(os.path.join(temp_path, filename), "wb") as image_file: # Open a new file in binary mode
      image_file.write(img_response.content)
    return image
  except:
    print("Erro ao tentar baixar: " + image["imgSrc"])
    return False

def download_images(q, limit=2000, url=None, images_downloaded=0, images_meta=[]):
  if len(os.listdir(temp_path)) > limit:
    with open(os.path.join(temp_path, "index.json"), "w") as json_file:
      json.dump(images_meta, json_file)
    return images_downloaded
  try:
    api_response = (requests.get(url=url) if url else requests.get(url="https://arquivo.pt/imagesearch", params={ "q": q })).json()
    images = api_response["responseItems"]
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
      future_images = {executor.submit(download_image, image): image for image in images}
      for future in concurrent.futures.as_completed(future_images):
        image = future.result()
        if image:
          images_meta.append(image)
          images_downloaded += 1
          print(image["imgSrc"] + " foi baixada.")
    if not api_response["responseItems"] or not api_response["nextPage"]:
      limit = 0
    return download_images(q, limit, api_response["nextPage"], images_downloaded, images_meta)
  except:
    print(f"Erro fatal ao tentar buscar: \"{url if url else q}\".")
    return images_downloaded
theme = "esoterismo"
print(f"Buscando imagens sobre o tema: {theme}")
print(f"Total de novas imagens baixadas: {download_images(theme)}")

Buscando imagens sobre o tema: esoterismo
http://www.jornaldascaldas.com/wp-content/uploads/2010/03/esoterismo1-150x150.jpg foi baixada.
http://t1.ftcdn.net/jpg/00/02/54/06/110_F_2540642_qtYEKR7IFqNqIliB8JPNynkikQX8IY.jpg foi baixada.
http://www.cristaisaquarius.com.br/media/wysiwyg/esot_rico2.jpg foi baixada.
http://www.esoterissima.com.br/wp-content/uploads/2017/09/ESOTERISMO.png foi baixada.
http://static-p3.fotolia.com/jpg/00/24/32/16/110_F_24321697_uE7RM5D3Vwk9D9dZLclPeo2U5vwiGk50.jpg foi baixada.
https://www.mediaxxi.com/wp-content/uploads/2016/11/O-esoterismo-100x100.jpg foi baixada.
https://img.cherchons.com/esoterisme.jpg foi baixada.
https://www.mediaxxi.com/wp-content/uploads/2016/11/O-esoterismo-600x905.jpg foi baixada.
http://www.bibliotheca.pt/79-cart_default/o-esoterismo-de-dante.jpg foi baixada.
https://cdn.portalsaofrancisco.com.br/wp-content/uploads/2016/07/esoterismo-013.jpg foi baixada.
http://www.bibliotheca.pt/81-cart_default/o-esoterismo-de-dante.jpg foi baixada.

In [26]:
import pandas as pd
with open(os.path.join(temp_path, "index.json"), "r") as json_file: # Open a new file in binary mode
  images_meta = json.load(json_file)
pd.DataFrame(images_meta[:5])

,imgDigest,imgSrc,pageTitle,imgHeight,imgWidth,imgMimeType,pageURL,imgTstamp,pageTstamp,imgCaption,imgAlt,imgTitle,collection,imgLinkToArchive,pageLinkToArchive
0,27a7bbf074b37b4c07dd58fe4e0361432107cc822c24d7...,http://www.jornaldascaldas.com/wp-content/uplo...,Jornal das Caldas. Edição On-line — Caldas da ...,150,150,image/jpeg,http://www.jornaldascaldas.com/index.php/page/3/,20100326205919,20100326205847,[Decorre até 31 de Março no Centro Comercial V...,[esoterismo],[esoterismo],[FAWP1],https://arquivo.pt/wayback/20100326205919im_/h...,https://arquivo.pt/wayback/20100326205847/http...
1,9d67635d4675f51af9a718681bd3edb33c84201a6ce0d7...,http://t1.ftcdn.net/jpg/00/02/54/06/110_F_2540...,Procurar fotos: Categoria: Ciência e Natureza ...,73,110,image/jpeg,http://pt.fotolia.com/cat2/40012000,20110127102258,20110125051842,[#2540642 XS - L | X Adicionar ao carrinho 244...,[esoterism],[esoterism],[AWP9],https://arquivo.pt/wayback/20110127102258im_/h...,https://arquivo.pt/wayback/20110125051842/http...
2,0a97365392d52a438636c841f3cd278dbd25c91bfa9ac0...,http://www.cristaisaquarius.com.br/media/wysiw...,Cristais Aquarius - A loja Esotérica para Comp...,100,1100,image/jpeg,http://www.cristaisaquarius.com.br/,20150813220641,20150813215429,[Cristais Super Limpos Ponta de Cristal de Alt...,[Esoterismo],[Esoterismo],[AWP18],https://arquivo.pt/wayback/20150813220641im_/h...,https://arquivo.pt/wayback/20150813215429/http...
3,6dff44fc733cde6532d75b98e47b4a79f0d546f427bb7c...,http://www.esoterissima.com.br/wp-content/uplo...,Esoterissima,450,700,image/png,http://www.esoterissima.com.br/,20190412033416,20190412033032,[ESOTERISMO Confira as notícias clicando na im...,NaN,[ESOTERISMO],[AWP29],https://arquivo.pt/wayback/20190412033416im_/h...,https://arquivo.pt/wayback/20190412033032/http...
4,c62b3f8349a86ab6f6aa8acc672d069a0c517032d36c8e...,http://static-p3.fotolia.com/jpg/00/24/32/16/1...,Procurar fotos: visão,110,83,image/jpeg,http://pt.fotolia.com/tag/vis%C3%A3o,20110122163603,20110121092741,[#24321697 XS - XL Adicionar ao carrinho 102111],[esotérisme],[esotérisme],[AWP9],https://arquivo.pt/wayback/20110122163603im_/h...,https://arquivo.pt/wayback/20110121092741/http...


4. Reúna dois colegas e elabore uma proposta de candidatura ao Prémio Arquivo pt 2024. Detalhe e explore, junto com os seus colegas, uma descrição sumária da ideia tendo em conta o seu impacto social e científico, a relevância da utilização do Arquivo.pt, originalidade e a exequibilidade da concretização do projeto. Para consultar os premiados das edições anteriores clique no seguinte link: https://sobre.arquivo.pt/pt/colabore/premios-arquivo-pt/